In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification')
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification/src')

In [5]:
from llm_lexical_simplifier import LLMLexicalSimplifier
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForMaskedLM
from simple_bert_lexical_simplifier import SimpleBertLexicalSimplifier
from benchmark_suite import BenchmarkSuite
from language import Language

Using cuda for model inference.


# LLM based

In [ ]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype="auto",
    trust_remote_code=True,
    # Flash attention is only supported starting Ampere architecture
    # So we cant use it on free-tier GPUs
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "do_sample": False,
}

# Define PAD Token = EOS Token
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [2]:
de_messages = [
    {"role": "user", "content": "Der Maschinenbau hat in Europa durch die Bildung der EU eine starke Erleichterung erhalten. Die vereinfachte Version des vorigen Satzes ist: Der Maschinenbau hat in Europa durch die Bildung der EU eine starke [MASK] erhalten. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Vereinfachung", "Entspannung", "Begünstigung", "Unterstützung", "Rückenwind,  "Förderung", "Rückhalt", "Aufwind", "Antrieb", "Erleichterung"]'},
    {"role": "user", "content": "Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen Sanktus signalisiert. Die vereinfachte Version des vorigen Satzes ist: Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen [MASK] signalisiert. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Zustimmung", "Einverständnis", "Okay", "Segen", "Billigung, Bestätigung, Bejahung, Zuspruch, Akzeptanz, Befürwortung"]'},
]
en_messages = [
    {"role": "user", "content": "The president’s relatives were also said to be critical of the way his closest advisers had prepared him for the debate. The simplified version of the previous sentence is: The president’s relatives were also said to be critical of the way his closest [MASK] had prepared him for the debate. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["staff", "team", "aides", "consultants", "assistants", "strategists", "supporters", "handlers", "counselors", "staffers"]'},
    {"role": "user", "content": "The intermediate sprint will come 94km or so into the stage. The simplified version of the previous sentence is: The intermediate sprint will come 94km or so into the [MASK]. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["segment", "part", "section", "phase", "leg", "interval", "course", "route", "portion", "strech"]'},
]
es_messages = [
    {"role": "user", "content": "Los esfuerzos del gobierno para reducir la deuda han sido aplaudidos por la comunidad internacional. La versión simplificada del anterior es: Los esfuerzos del gobierno para reducir la deuda han sido [MASK] por la comunidad internacional. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["reconocidos", "elogiados", "valorados", "aprobados", "aceptados", "apreciados", "celebrados", "alabados", "felicitados, "notado"]'},
    {"role": "user", "content": "La reunión entre los líderes fue descrita como un éxito por ambas partes. La versión simplificada del anterior es: La reunión entre los líderes fue descrita como un [MASK] por ambas partes. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["éxito", "logro", "triunfo", "acierto", "avance", "progreso", "éxito rotundo", "consecución", "gran paso", "éxito importante"]'},
]

pt_messages = [
    {"role": "user", "content": "A nova legislação tem como objetivo melhorar a segurança no trabalho. A versão simplificada do anterior é: A nova legislação tem como objetivo melhorar a [MASK] no trabalho. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["segurança", "proteção", "condição", "segurança ocupacional", "salubridade", "qualidade", "segurança laboral", "condições de trabalho", "prevenção", "resguardo"]'},
    {"role": "user", "content": "O projeto de infraestrutura deve gerar milhares de empregos. A versão simplificada do anterior é: O projeto de infraestrutura deve gerar milhares de [MASK]. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["empregos", "vagas", "trabalhos", "oportunidades", "postos", "cargos", "colocações", "funções", "contratações", "empregos diretos"]'}
]

llm_ls = LLMLexicalSimplifier(model=model, tokenizer=tokenizer,
                              pattern='{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}. Ten results as ordered, AST parsable Python list:',
                              exemplars=de_messages, mask_token='[MASK]', generation_args=generation_args)

NameError: name 'LLMLexicalSimplifier' is not defined

In [ ]:
llm_output = llm_ls.generate_substitutions_for('heikel', 'Die politische Lage in Syrien ist heikel.')
llm_output

['komplex',
 'instabil',
 'schwer',
 'spannend',
 'heikel',
 'kompliziert',
 'wichtig',
 'sensibel',
 'sensationell',
 'aufregend']

In [ ]:
suite = BenchmarkSuite(llm_ls, {
        Language.EN: {'pattern': '{original_sentence}. The simplified version of the previous sentence is: '
                                 '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
                                 'Python list:',
                      'exemplars': en_messages},
        Language.DE: {'pattern': '{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: '
                                 '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
                                 'Python list:',
                      'exemplars': de_messages}})
suite.run()

Benchmarking model on DE ...
Benchmarking model on GermanEvalDataProvider...


Benchmarking:   0%|          | 2/1040 [00:09<1:26:09,  4.98s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  The Maschinenbau, or the machine-building industry, has indeed benefited significantly from the formation of the European Union (EU). The simplification of the sentence while maintaining the essence of the original statement can be achieved by identifying synonyms or phrases that convey the same meaning of'strong support' or'strong facilitation'. Here are the results in AST parsable Python list format:

```python
["Vereinfachung", "
Returning empty list.


Benchmarking:   0%|          | 3/1040 [00:16<1:39:39,  5.77s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  ["Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung", "Einführung",
Returning empty list.


...

Benchmarking:  99%|█████████▊| 1026/1040 [1:15:43<01:10,  5.01s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  ["unvermittelte", "plötzliche", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwart
Returning empty list.


Benchmarking:  99%|█████████▉| 1030/1040 [1:16:04<00:53,  5.37s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  ["unvermittelte", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwartete", "unerwart
Returning empty list.


Benchmarking: 100%|██████████| 1040/1040 [1:16:49<00:00,  4.43s/it]


#BERT based

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
bert_model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-cased")

In [ ]:
bert_ls = SimpleBertLexicalSimplifier(bert_model, bert_tokenizer, '{original_sentence}. The simplified version of the previous sentence is: {sentence_with_complex_word_masked}. Top ten results:', None, '[MASK]')

In [ ]:
suite = BenchmarkSuite(bert_ls,
                       {Language.EN: {'pattern': '{original_sentence}. The simpler version of the previous sentence is: {sentence_with_complex_word_masked}. Top ten results:', 'exemplars': None},
                        Language.DE: {'pattern': '{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}. Die zehn besten Resultate:', 'exemplars': None},
                        Language.ES: {'pattern': '{original_sentence}. La versión simplificada de la frase anterior es: {sentence_with_complex_word_masked}. Los diez primeros resultados:', 'exemplars': None},
                        Language.PT: {'pattern': '{original_sentence}. A versão simplificada da frase anterior é: {sentence_with_complex_word_masked}. Os dez melhores resultados:', 'exemplars': None}})
suite.run()